In [23]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/final_cnv.csv',index_col=0)
train.head()

,cell_name,A1BG,A1BG-AS1,A1CF,A2M,A2ML1,A2MP1,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,...,ZRANB3,ZSCAN1,ZSCAN2,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSCAN10,ZSCAN12,ZSCAN12P1,ZSCAN16,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,AU565,0,0,-1,1,1,1,-1,-1,1,-1,0,1,1,-1,-1,-1,-1,-1,1,1,1,-1,1,1,-1,-1,-1,0,1,1,-1,-1,1,-1,-1,1,1,1,1,...,-1,0,-1,0,0,0,-1,1,-1,-1,-1,0,-1,1,0,-1,1,-1,1,-1,-1,1,1,-1,1,-1,-1,-1,-1,1,-1,1,-1,1,-1,-1,-1,1,-1,0
1,BT-20,1,1,0,1,1,1,-1,1,0,0,1,-1,-1,0,0,0,1,1,1,1,-1,0,1,1,1,1,0,0,1,1,1,-1,0,1,1,0,0,0,0,...,1,-1,1,-1,1,1,1,0,1,1,1,-1,0,0,1,1,0,1,1,-1,1,0,1,1,1,-1,0,-1,1,-1,1,1,1,0,1,0,0,1,1,0
2,BT-474,0,0,0,1,1,1,-1,1,0,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,0,-1,1,1,1,0,-1,-1,-1,1,2,-1,0,-1,0,0,-1,2,2,...,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,1,1,1,2,-1,-1,-1,-1,0,-1,-1,1,1,-1,-1,1,-1,0
3,BT-483,0,0,0,1,1,1,-1,1,1,1,0,1,1,-1,-1,0,0,-1,-1,-1,-1,0,1,2,0,0,0,0,-1,1,0,0,0,0,0,0,-1,1,1,...,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,-1,0,0,1,-1,1,-1,-1,0,-1,-1,-1,-1,-1,0,1,-1,-1,2,-1,-1
4,BT-549,-1,-1,0,0,0,0,-1,0,-1,-1,1,0,0,1,1,-1,1,-1,-1,0,-1,-1,1,1,-1,1,-1,-1,0,0,-1,-1,-1,1,1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,1,1,1,-1,1,0,-1,1,0,1,-1,0,1,-1,1,-1,1,-1,1,1,1,0,1,0,-1,0,0,1,1,1,1,1


In [26]:
target=train['cell_name']

In [27]:
train.drop(['cell_name'], axis=1, inplace=True)
train.head()

,A1BG,A1BG-AS1,A1CF,A2M,A2ML1,A2MP1,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,...,ZRANB3,ZSCAN1,ZSCAN2,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSCAN10,ZSCAN12,ZSCAN12P1,ZSCAN16,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0,0,-1,1,1,1,-1,-1,1,-1,0,1,1,-1,-1,-1,-1,-1,1,1,1,-1,1,1,-1,-1,-1,0,1,1,-1,-1,1,-1,-1,1,1,1,1,-1,...,-1,0,-1,0,0,0,-1,1,-1,-1,-1,0,-1,1,0,-1,1,-1,1,-1,-1,1,1,-1,1,-1,-1,-1,-1,1,-1,1,-1,1,-1,-1,-1,1,-1,0
1,1,1,0,1,1,1,-1,1,0,0,1,-1,-1,0,0,0,1,1,1,1,-1,0,1,1,1,1,0,0,1,1,1,-1,0,1,1,0,0,0,0,-1,...,1,-1,1,-1,1,1,1,0,1,1,1,-1,0,0,1,1,0,1,1,-1,1,0,1,1,1,-1,0,-1,1,-1,1,1,1,0,1,0,0,1,1,0
2,0,0,0,1,1,1,-1,1,0,1,0,1,1,-1,-1,-1,-1,-1,-1,-1,0,-1,1,1,1,0,-1,-1,-1,1,2,-1,0,-1,0,0,-1,2,2,-1,...,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,1,1,1,2,-1,-1,-1,-1,0,-1,-1,1,1,-1,-1,1,-1,0
3,0,0,0,1,1,1,-1,1,1,1,0,1,1,-1,-1,0,0,-1,-1,-1,-1,0,1,2,0,0,0,0,-1,1,0,0,0,0,0,0,-1,1,1,-1,...,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,-1,0,0,1,-1,1,-1,-1,0,-1,-1,-1,-1,-1,0,1,-1,-1,2,-1,-1
4,-1,-1,0,0,0,0,-1,0,-1,-1,1,0,0,1,1,-1,1,-1,-1,0,-1,-1,1,1,-1,1,-1,-1,0,0,-1,-1,-1,1,1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,1,1,1,-1,1,0,-1,1,0,1,-1,0,1,-1,1,-1,1,-1,1,1,1,0,1,0,-1,0,0,1,1,1,1,1


In [28]:
train_scaled = minmax_scale(train, axis = 0)

In [29]:
ncol = train_scaled.shape[1]
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))

In [30]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Deep Autoencoder

features_path = '/content/drive/MyDrive/AE/deep_autoe_features.pickle'
labels_path = '/content/drive/MyDrive/AE/deep_autoe_labels.pickle'

# this is the size of our encoded representations
encoding_dim = 32

# this is our input placeholder; 784 = 28 x 28
input = Input(shape = (ncol, ))

my_epochs = 50

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim * 4, activation='relu')(input)
encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
decoded = Dense(encoding_dim * 4, activation='relu')(decoded)
decoded = Dense(ncol, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input, decoded)

# Separate Encoder model

# this model maps an input to its encoded representation
encoder = Model(input, encoded)

# Separate Decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))
# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Train to reconstruct MNIST digits

# configure model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# prepare input data
#(x_train, _), (x_test, y_test) = mnist.load_data()
x_train, x_test, y_train, y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))
# normalize all values between 0 and 1 and flatten the 28x28 images into vectors of size 784
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# Train autoencoder for 50 epochs

autoencoder.fit(x_train, x_train, epochs=my_epochs, batch_size=8, shuffle=True, validation_data=(x_test, x_test),
                verbose=2)

# after 100 epochs the autoencoder seems to reach a stable train/test lost value

# Visualize the reconstructed encoded representations

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# save latent space features 32-d vector
# pickle.dump(encoded_imgs, open(features_path, 'wb'))
# pickle.dump(y_test, open(labels_path, 'wb'))

# n = 10  # how many digits we will display
# plt.figure(figsize=(10, 2), dpi=100)
# for i in range(n):
#     # display original
#     ax = plt.subplot(2, n, i + 1)
#     plt.imshow(x_test[i].reshape(28, 28))
#     plt.gray()
#     ax.set_axis_off()

#     # display reconstruction
#     ax = plt.subplot(2, n, i + n + 1)
#     plt.imshow(decoded_imgs[i].reshape(28, 28))
#     plt.gray()
#     ax.set_axis_off()

# plt.show()

encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train.head()

K.clear_session()

(41, 20669)
(5, 20669)
Epoch 1/50
6/6 - 1s - loss: 0.6932 - val_loss: 0.6933 - 884ms/epoch - 147ms/step
Epoch 2/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 263ms/epoch - 44ms/step
Epoch 3/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 268ms/epoch - 45ms/step
Epoch 4/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 248ms/epoch - 41ms/step
Epoch 5/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 256ms/epoch - 43ms/step
Epoch 6/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 261ms/epoch - 43ms/step
Epoch 7/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 253ms/epoch - 42ms/step
Epoch 8/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 264ms/epoch - 44ms/step
Epoch 9/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 270ms/epoch - 45ms/step
Epoch 10/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 255ms/epoch - 43ms/step
Epoch 11/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 256ms/epoch - 43ms/step
Epoch 12/50
6/6 - 0s - loss: 0.6932 - val_loss: 0.6933 - 259ms/epoch - 43ms/step
Epoch 13/50
6

In [31]:
encoded_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,0.000000,0.0,1.925533,0.368401,0.154952,0.209214,0.617154,0.503612,0.0,0.723861,0.269678,0.240563,1.002840,0.808006,0.850878,0.289126,0.0,0.618140,0.000000,0.300096,0.168188,0.000000,0.000000,0.292417,1.469174,0.0,0.000000,0.273416,0.104517,0.348120,0.501566,0.625033
1,0.000000,0.0,2.076089,0.000000,0.000000,0.269225,0.583290,0.417489,0.0,0.292712,0.465527,0.331685,0.883986,0.117826,0.562825,0.840661,0.0,0.582957,0.212668,0.000000,0.000000,0.000000,0.275182,0.501334,1.126365,0.0,0.036576,0.690393,0.371611,0.000000,0.902201,0.593244
2,0.000000,0.0,1.314588,0.000000,0.002116,0.149147,0.829237,0.366385,0.0,0.407778,0.344011,0.246295,0.913877,0.177209,0.690450,0.289573,0.0,0.222045,0.321030,0.425434,0.086859,0.000000,0.536774,0.675671,0.745308,0.0,0.158403,0.839465,0.818558,0.168659,0.438294,0.365900
3,0.226834,0.0,1.457882,0.000000,0.639746,0.564666,0.477892,0.566226,0.0,0.482649,0.474974,0.000000,0.024408,0.447510,0.979686,0.304000,0.0,0.662194,0.000000,0.000000,0.462305,0.000000,0.000000,0.390726,0.620660,0.0,0.305250,0.000000,0.210320,0.362644,0.548815,0.408334
4,0.000000,0.0,1.869834,0.000000,0.114000,0.000000,1.044253,0.584714,0.0,0.698169,0.575220,0.482129,0.419910,0.242070,0.093087,0.523987,0.0,0.772430,0.000000,0.000000,0.021694,0.000000,0.000000,0.532934,0.583713,0.0,0.449748,0.571713,0.000000,0.000000,0.835864,0.535595
5,0.000000,0.0,1.546005,0.000000,0.077604,0.000000,0.921053,0.481562,0.0,0.509215,0.264428,0.000000,0.341712,0.000000,0.347922,0.649737,0.0,0.583126,0.048801,0.249424,0.000000,0.000000,0.084652,0.843347,0.986673,0.0,0.000000,0.035115,0.386027,0.000000,0.989464,0.681513
6,0.000000,0.0,1.433569,0.000000,0.489951,0.288110,0.140764,0.779924,0.0,0.372942,0.425381,0.453363,0.757400,0.522846,0.560475,0.391150,0.0,0.330176,0.000000,0.000000,0.111956,0.000000,0.256963,0.389619,0.805842,0.0,0.367197,0.484147,0.160041,0.000000,0.488096,0.661889
7,0.000000,0.0,1.443448,0.000000,0.109986,0.202613,0.316271,0.492627,0.0,0.463011,0.341307,0.357802,0.534078,0.307956,0.529678,0.464433,0.0,0.584729,0.301605,0.269029,0.000000,0.000000,0.019519,0.125830,0.602115,0.0,0.034088,0.580995,0.196575,0.000000,0.611165,0.319745
8,0.000000,0.0,1.946357,0.000000,0.000000,0.553585,0.136034,0.502221,0.0,0.312971,0.685280,0.085630,0.596272,0.000000,0.946333,1.032483,0.0,0.620496,0.111970,0.528513,0.219822,0.000000,0.242877,0.364455,1.629547,0.0,0.000000,0.321559,0.514778,0.000000,0.743745,0.176979
9,0.000000,0.0,1.237150,0.000000,0.860024,0.206836,0.578451,0.781518,0.0,0.458093,0.212657,0.469542,0.133733,0.821224,0.237388,0.000000,0.0,0.741468,0.000000,0.000000,0.000000,0.000000,0.000000,0.035177,0.505667,0.0,0.346671,0.560721,0.000000,0.139696,0.606897,0.358759


In [32]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20669)]           0         
                                                                 
 dense (Dense)               (None, 128)               2645760   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 20669)             266630

In [33]:
encoded_train = encoded_train.loc[:, (encoded_train != 0).any(axis=0)]
encoded_train = encoded_train.drop(columns=encoded_train.columns[encoded_train.eq(0).mean()> 0.8])
encoded_train.shape

(46, 25)

In [36]:
encoded_train.to_excel('/content/drive/MyDrive/Datasets/dae_cnv.xlsx', index = False)